In [1]:
using LinearAlgebra
using LsqFit
using LaTeXStrings
using Plots
using Statistics
using ProgressMeter
using JLD

global const rho::Float64 = (sqrt(5) - 1) / 2  # 1 / phi
global const rho2::Float64 = (3 - sqrt(5)) / 2;  # 1 / phi^2

In [2]:
function build_JK(ns, A=50)
    N = 4 + 3*(ns - 1)
    
    J = zeros(N, N)
    K = zeros(N)
    
    J[1, ns+1] += 2
    J[2*ns+1, N] += 2
    
    # source and target horizontal edges
    J[1, 2] += -1
    J[1, ns+2] += -1
    K[1] += -2
    
    J[2*ns, N] += -1
    J[N-1, N] += -1
    K[N] += -2

    # horizontal corner edges
    J[ns, 2*ns] += 1
    J[ns-1, ns] += -1
    J[ns, 2*ns+1] += -1
    K[ns] += 2
    
    J[ns+2, 2*ns+2] += 1
    J[2*ns+2, 2*ns+3] += -1
    J[ns+1, 2*ns+2] += -1
    K[2*ns+2] += 2
    
    # vertical corner edges    
    J[ns+1, 2*ns+2] += -1
    J[ns, 2*ns+1] += -1
    
    # lower horizontal internal edges
    for i in 2:ns-1
        J[i, i+ns] += 1
        J[i-1, i] += -1
        J[i, i+ns+1] += -1
        J[i, i+1] += -1
    end
    # vertical internal edges
    for i in ns+2:2*ns
        J[i-ns, i] += 1
        J[i, i+ns] += 1
        J[i-ns-1, i] += -1
        J[i, i+ns+1] += -1
        K[i] += 4
    end
    # higher horizontal internal edges
    for i in 2*ns+3:N-1
        J[i-ns, i] += 1
        J[i-1, i] += -1
        J[i-1-ns, i] += -1
        J[i, i+1] += -1
    end
    
    K = A/4 * K
    J = A/4 * J
    J = J .+ transpose(J)

    J, K
end;

In [3]:
function compute_delta(Nd::Int, s::Float64, V, K, EJ)
    Heff = zeros(Nd+1, Nd+1)
    for j in 1:Nd
        Heff[1, j+1] = s*sum(K .* V[:, j])
        Heff[j+1, 1] = s*sum(K .* V[:, j])

        Heff[j+1, j+1] = 2*(1-s) + s*EJ
    end
    
    E = eigvals(Heff)[1:2]
    return E[2] - E[1]
end


function find_delta_min(Nd::Int, V, K, EJ, tol=1e-4::Float64)
    a = 0
    b = 0.5

    h = b - a
    c = a + rho2 * h
    d = a + rho * h
    
    # Required steps to achieve tolerance
    n = trunc(Int, ceil(log(tol / h) / log(rho)))

    yc = compute_delta(Nd, c, V, K, EJ)
    yd = compute_delta(Nd, d, V, K, EJ)
    
    for k in 1:n
        if yc < yd
            b = d
            d = c
            yd = yc
            h = rho * h
            c = a + rho2 * h

            yc = compute_delta(Nd, c, V, K, EJ)
        else
            a = c
            c = d
            yc = yd
            h = rho * h
            d = a + rho * h

            yd = compute_delta(Nd, d, V, K, EJ)
        end
    end
    
    if yc < yd
        sc = (a + d) / 2
    else
        sc = (b + c) / 2
    end
    
    dmin = compute_delta(Nd, sc, V, K, EJ)
    
    return sc, dmin
end;

In [4]:
ns = 1
N = 4

B = 1
A = N*B

J1 = zeros(N, N)
J1[1, 2] = A/2
J1[3, 4] = A/2
J1[1, 3] = -A/2
J1[2, 4] = -A/2

J1 = J1 + transpose(J1)
K1 = zeros(N);

In [26]:
M = 100000

ns_max = 15

sc = zeros(ns_max, 2)
dmin = zeros(ns_max, 2)
for ns in 1:ns_max
    A = N*B
    
    N = 4 + 3*(ns - 1)

    if ns == 1
        J, K0 = deepcopy(J1), deepcopy(K1)
    else
        J, K0 = build_JK(ns, A)
    end

    E, V = eigen(J)
    E = round.(E, digits=3)
    Nd = size(E[abs.(E .- E[1]) .== 0])[1]
    EJ = E[1]
    
    scM = zeros(M)
    dminM = zeros(M)
    W = rand(N, M)
    @showprogress for j in 1:M
        K = zeros(N)
        for (i, w) in enumerate(W[:, j])
            K[i] = K0[i] + B*w/2
        end

        scM[j], _ = find_delta_min(Nd, V, K, EJ)
        
        paths = zeros(0)
        for e in 1:ns-1
            path = sum([W[i, j] for i in 1:e]) + W[e+ns+1, j] + sum([W[i, j] for i in e+2*(ns+1):3*ns+1])
            append!(paths, path)
        end

        append!(paths, sum([W[i, j] for i in 1:ns]) + W[2*ns+1, j])
        append!(paths, sum([W[i, j] for i in 2*ns+2:3*ns+1]) + W[ns+1, j])

        paths = sort(paths)
        df = paths[2] - paths[1]
        
        dminM[j] = df * scM[j]
    end

    sc[ns, :] = [mean(scM), std(scM) / sqrt(M)]
    dmin[ns, :] = [mean(dminM), std(dminM) / sqrt(M)]
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
Progress: 100%|█████████████████████████████████████████| Time: 0:00:15
Progress: 100%|█████████████████████████████████████████| Time: 0:00:19
Progress: 100%|█████████████████████████████████████████| Time: 0:00:22
Progress: 100%|█████████████████████████████████████████| Time: 0:00:26
Progress: 100%|█████████████████████████████████████████| Time: 0:00:30
Progress: 100%|█████████████████████████████████████████| Time: 0:00:34
Progress: 100%|█████████████████████████████████████████| Time: 0:00:41
Progress: 100%|█████████████████████████████████████████| Time: 0:00:49
Progress: 100%|█████████████████████████████████████████| Time: 

In [27]:
using JLD

save("data/A=N/dminEff.jld", "sc", sc, "dmin", dmin)